In [1]:
# -*- coding: utf8 -*-
import pandas as pd
import numpy as np
import sys
import scipy.stats as sps
from scipy.optimize import brentq
NORM_CDF = sps.norm.cdf
NORM_PDF = sps.norm.pdf

In [2]:
sys.path.append('../')
import APICenter as API
import DataCenter as DATA
Api=API.API()
TestModel=True
if TestModel:
    Data=DATA.TestDataCenter()
else:
    Data=DATA.DataCenter()
import OptionPricing as OP  
#load function in current path
IV_LOWER_BOUND = 1e-8

In [17]:

###################################################################
#K : option strike price
#S : price of the underlying asset
#r : risk-free interest rate, 0.03 as 3%
#sigma : asset volatility or implied volatility, 0.25 as 25%
#T : days before expiry, 31/365 as 31 days

#https://en.wikipedia.org/wiki/Black%E2%80%93Scholes_model#Black-Scholes_formula
#In practice, some sensitivities are usually quoted in scaled-down terms,
# to match the scale of likely changes in the parameters. For example,
# rho is often reported divided by 10,000 (1 basis point rate change),
# vega by 100 (1 vol point change),
# theta by 365 or 252
# (1 day decay based on either calendar days or trading days per year).
class Sensibility():
    # Theta 时间de导数
    # rho   利率的导数
    # Delta是option价值对于股票的导数，
    # gamma是delta对于股票的导数
    # vega是option价格对于波动性的导数
    # --------------
    # theta，rho可以跨不同股票相加
    # delta，gamma只能相同股票的相加
    # vega可以同股票相加，但如果需要精确的exposure应该用vega map，
    # 也就是每一个股票的每一个option maturity的每一个strike都有自己的vega。
    
    #Delta（Δ），Gamma（γ），Theta（θ），Vega（ν），Rho（ρ）
    def __init__(self,Model='Black-Sholes',OptionStyle='European'):
        self.Model=Model
        self.OptionStyle=OptionStyle
        """self.Delta=Delta()
        self.Gamma=Gamma()
        self.Theta=Theta()
        self.Vega=Vega()
        self.Rho=Rho()"""
        pass
    def getGreeks(self,UnderlyingPrice,
                        Strike,
                        Volatility,
                        Time2Maturity,
                        DividendYield,
                        RiskFreeRate,
                        OptionType):
        s=UnderlyingPrice
        k=Strike
        sigma=Volatility
        t=Time2Maturity
        g=DividendYield
        r=RiskFreeRate
        self.OptionType=OptionType
        Delta,Gamma,Vega,Theta,Rho=self.Delta(s,k,r,g,sigma,t),self.Gamma(s,k,r,sigma,t),self.Vega(s,k,r,sigma,t),self.Theta(s,k,r,sigma,t),self.Rho(s,k,r,sigma,t)
        return Delta,Gamma,Vega,Theta,Rho
    def BlackScholes_d1(self,s, k, r, sigma, t):
        d1=(np.log(s/k)+((r-g)+0.5*(sigma)**2)*(t))/(sigma*np.sqrt(t))
        return d1
    def BlackScholes_d2(self,S, K, r, sigma, T):
        #return (LOG_F(S/K) + (r - sigma**2 / 2) * T) / (sigma * np.sqrt(T))
        return self.BlackScholes_d1(S, K, r, sigma, T) - (sigma * np.sqrt(T))
    def Delta(self,s,k,r,g,sigma,t):#S, K, r,g, sigma, T
        #Delta（Δ）又称对冲值 :option价值对于股票的导数，
        #dC/dS
        if self.Model=='Black-Sholes' and self.OptionStyle=='European':
            d1=(np.log(s/k)+((r-g)+0.5*(sigma)**2)*(t))/(sigma*np.sqrt(t))
            if self.OptionType in ('Call','call','C','c'):
                deltavalue=sps.norm.cdf(d1)
            elif self.OptionType in ('Put','put','P','p'):
                #d1=(np.log(s/k)+((r-g)+0.5*(sigma)**2)*(t))/(sigma*np.sqrt(t))
                #d2=d1-sigma*np.sqrt(t)
                deltavalue=sps.norm.cdf(d1)-1
            else:
                raise ValueError('Unknown Option type: %s'%OptionType)
        else:
            raise ValueError('Unsupported Model or OptionStyle:(%s,%s), Waiting for upgrading.'%(Model,OptionStyle))
        return deltavalue
        
    def Gamma(self,S,K,r,sigma,T):#S, K, r, sigma, T
        # Gamma（γ）Delta对于股票的导数
        # d2C/dS2
        
        if sigma > IV_LOWER_BOUND:
            return sps.norm.pdf(self.BlackScholes_d1(S, K, r, sigma, T)) / (S * sigma * np.sqrt(T))
        return 0
    def Vega(self,S,K,r,sigma,T):#S, K, r, sigma, T
        # Vega（ν）option价格对于波动性的导数
        
        
        return sps.norm.pdf(self.BlackScholes_d1(S, K, r, sigma, T)) * S * np.sqrt(T)
    def Theta(self,S,K,r,sigma,T):#S, K, r, sigma, T
        # Theta（θ）时间de导数
        
        if self.OptionType in ('Call','call','C','c'):
            
            return (-S * sigma * sps.norm.pdf(self.BlackScholes_d1(S, K, r, sigma, T)) / (2 * np.sqrt(T)) -
                    r * K * np.exp(-r * T) * sps.norm.cdf(self.BlackScholes_d2(S, K, r, sigma, T)))
        elif self.OptionType in ('Put','put','P','p'):
            return (-S * sigma * sps.norm.pdf(self.BlackScholes_d1(S, K, r, sigma, T)) / (2 * np.sqrt(T)) +
                r * K * np.exp(-r * T) * sps.norm.cdf(-self.BlackScholes_d2(S, K, r, sigma, T)))
        else:
            raise ValueError('Unknown Option type: %s'%OptionType)

    
    def Rho(self,S,K,r,sigma,T):
        # Rho（ρ）利率的导数
        
        if self.OptionType in ('Call','call','C','c'):
            return K * T * np.exp(-r * T) * sps.norm.cdf(self.BlackScholes_d2(S, K, r, sigma, T))
        elif self.OptionType in ('Put','put','P','p'):
            return -K * T * np.exp(-r * T) * sps.norm.cdf(-self.BlackScholes_d2(S, K, r, sigma, T))
        else:
            raise ValueError('Unknown Option type: %s'%OptionType)


In [20]:
Greek=Sensibility()
Delta,Gamma,Vega,Theta,Rho=Greek.getGreeks(UnderlyingPrice=2.634,
                        Strike=2.45,
                        Volatility=.23,
                        Time2Maturity=18.0/252,
                        DividendYield=0,
                        RiskFreeRate=0.04,
                        OptionType='Call')

Delta,Gamma,Vega,Theta,Rho

S,K,r,T,sigma,cp_sign,g=2.634,2.45,0.04,18.0/252,.23,-1,0
cp=1.00998
sigma,status,ratio=OP.ImpliedVolatility().ImpliedVolatility_OlD(S=2.634,
                                                                K=2.45,
                                                                r=0.04,
                                                                T=18.0/252,
                                                                cp=1.00998,
                                                                cp_sign=-1,
                                                                g=0)
print ratio*cp
sigma,status,ratio

(0.8953109560616472,
 1.120642142538341,
 0.12773158225968928,
 -0.2920056433131427,
 0.15421034977686243)